In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy.special.orthogonal import p_roots
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SQuad
from poly import lagrange_list
from basis import LagrangeBasisQuad
from mesh import Mesh, uniform_nodes_2d
from assemble import simple_assembly, simple_build_rhs
from poisson import poisson_Kloc

## Mesh nodes

In [ ]:
order   = 2
L       = 1.0
n_elems = 16

x_max = L
y_max = L

vertices, elem_to_vertex, boundary_vertices = \
         uniform_nodes_2d(n_elems, x_max, y_max)

## Assembly

In [ ]:
topo  = SQuad()
basis = LagrangeBasisQuad(topo, order)
mesh  = Mesh(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

nodes = vertices[elem_to_vertex]
jacb  = topo.calc_jacb(nodes)
jacb_det = topo.calc_jacb_det(jacb)
jacb_inv = topo.calc_jacb_inv(jacb)
jacb = jacb[0]
jacb_det = jacb_det[0]
jacb_inv = jacb_inv[0]

In [ ]:
Kloc = poisson_Kloc(basis, jacb_det, jacb_inv)
K = simple_assembly(mesh, Kloc)

def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,2))
    x = X[:,0]
    y = X[:,1]
    return (x*(x-x_max)*y*(y-y_max)).reshape(shape)

def f2(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,2))
    x = X[:,0]
    y = X[:,1]
    return (2*y*(y-y_max)+x*(x-x_max)*2).reshape(shape)

rhs = simple_build_rhs(topo, basis, mesh, f2)

In [ ]:
plt.spy(K)
(K-K.T).nnz

In [ ]:
ml = ruge_stuben_solver(K)
residuals = []
sol = ml.solve(rhs, tol=1e-12, residuals=residuals, maxiter=5000)
len(residuals), residuals[-1]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

dof_phys = mesh.get_dof_phys()
X = dof_phys[:,0]
Y = dof_phys[:,1]
Z1 = sol
Z2 = f(dof_phys)
ax.plot_wireframe(X, Y, Z1)
ax.plot_wireframe(X, Y, Z2, 
                  color='g')
np.max(np.abs(Z1-Z2))